### III- Random forest

Le modèle

In [31]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import sys
import os

print("Current working directory:", os.getcwd())

sys.path.append(os.path.abspath('..'))
print("Updated sys.path:", sys.path)
from fonctions import defined_X_y
from sklearn.ensemble import RandomForestClassifier
from skopt.space import Real,Integer,Categorical
from skopt import BayesSearchCV
from timeit import default_timer as timer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

Current working directory: c:\Users\lulil\OneDrive\Documents\SCORING-TIDE\modélisation
Updated sys.path: ['c:\\Users\\lulil\\OneDrive\\Documents\\SCORING-TIDE\\modélisation', 'c:\\Users\\lulil\\anaconda3\\python39.zip', 'c:\\Users\\lulil\\anaconda3\\DLLs', 'c:\\Users\\lulil\\anaconda3\\lib', 'c:\\Users\\lulil\\anaconda3', '', 'c:\\Users\\lulil\\anaconda3\\lib\\site-packages', 'c:\\Users\\lulil\\anaconda3\\lib\\site-packages\\win32', 'c:\\Users\\lulil\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\Users\\lulil\\anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\Users\\lulil\\OneDrive\\Documents\\SCORING-TIDE', 'c:\\Users\\lulil\\OneDrive\\Documents\\SCORING-TIDE']


In [32]:
table = pq.read_table('../data/train_dummies_data.parquet')
t = pq.read_table('../data/test_dummies_data.parquet')
train_df = table.to_pandas()
test_df = t.to_pandas()

In [3]:
X, y = defined_X_y(train_df, 'fraude')

In [6]:
param_dict_bayes_RF = {
    'n_estimators' : Integer(10,200),
    'criterion': Categorical(['gini', 'entropy']), 
    'max_depth': Integer(2,10),
    'bootstrap':Categorical([True,False])}

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from skopt import BayesSearchCV

In [25]:
df_sample = train_df.sample(frac=0.1, random_state=42)

In [26]:
X_tr, y_tr = defined_X_y(df_sample, 'fraude')

In [24]:
#recherche des hyperparèmetres optimaux

opt = BayesSearchCV(RandomForestClassifier(),param_dict_bayes_RF,n_iter=5,cv=3,scoring = 'roc_auc',random_state=1)


opt.fit(X_tr, y_tr)
best_param_opti_bayes =opt.best_params_
best_score_opti_bayes = opt.best_score_
all_result_opti_bayes = opt.cv_results_
print(" #### RECHERCHE DES PARAMETRES OPTIMAUX POUR LE RANDOM FOREST #### ")
print("\n Paramètres optimaux : ")
best_param_opti_bayes =opt.best_params_
print(best_param_opti_bayes)
best_score_opti_bayes = opt.best_score_
print("\n Résultats  : " + str(best_score_opti_bayes))

 #### RECHERCHE DES PARAMETRES OPTIMAUX POUR LE RANDOM FOREST #### 

 Paramètres optimaux : 
OrderedDict([('bootstrap', False), ('criterion', 'gini'), ('max_depth', 8), ('n_estimators', 63)])

 Résultats  : 0.817093645845229


In [28]:
rf_clf = RandomForestClassifier(**opt.best_params_)
rf_clf.fit(X_tr,y_tr)

RandomForestClassifier(bootstrap=False, max_depth=8, n_estimators=63)

In [30]:
importance_variable_rf = pd.DataFrame()
importance_variable_rf["Variable"] = X_tr.columns
importance_variable_rf["Feature Importance"] = rf_clf.feature_importances_

# Sort the DataFrame by feature importance
importance_variable_rf = importance_variable_rf.sort_values(by="Feature Importance", ascending=False)

# Display the DataFrame
print(importance_variable_rf)

                     Variable  Feature Importance
2    fm_velocity_condition_24            0.188994
3                   fm_sum_24            0.110427
1                     montant            0.070200
4        fm_redondance_mcc_24            0.043125
154                  pays_792            0.042584
..                        ...                 ...
352                  mcc_3517            0.000000
353                  mcc_3518            0.000000
354                  mcc_3519            0.000000
355                  mcc_3520            0.000000
438                  mcc_3668            0.000000

[876 rows x 2 columns]


<div class="alert alert-block alert-info">
<b>Rappel:</b> Deux approches pour sélectionner un modèle et le seuil :
 <br>1. Approche statistique : maximiser un critère tel que l'AUC
 <br>2. Approche métier : arbitrer utiliser le lift
</div>

In [ ]:
fpr_test_RF, tpr_test_RF, _ = roc_curve(y_test, rf_clf.predict_proba(X_test)[:,1])
roc_auc_test_RF = auc(fpr_test_RF, tpr_test_RF)

In [ ]:
# ROC

param_graph()
lw = 2

# Random Forest
plt.plot(fpr_test_RF, tpr_test_RF, color='darkorange',
         lw=lw, label='RF - ROC curve (area = %0.2f)' % roc_auc_test_RF)



plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Comparaison courbes ROC (TEST)')
plt.legend(loc="lower right")
plt.show()